# 問題1 機械翻訳の実行とコードリーディング

In [43]:
#ライブラリのインポート
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np



#ハイパーパラメータの設定
batch_size = 64  # Batch size for training.
epochs = 10  # Number of epochs to train for.
latent_dim = 256  # Latent dimensionality of the encoding space.
num_samples = 10000  # Number of samples to train on.


# テキストデータファイルの読み込み
data_path = '/Users/shigaryouta/Downloads/fra-eng/fra.txt'

#ベクトル化した文を格納するリストを定義
# Vectorize the data.
input_texts = []
target_texts = []
#文章の中に存在する単語を格納するsetを定義
input_characters = set()
target_characters = set()
#テキストファイルを読み込み文章を抽出
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')
    
#文章から文を抽出し,for文で回す
for line in lines[: min(num_samples, len(lines) - 1)]:
    #作成した文を格納するリストに文を格納
    input_text, target_text, _ = line.split('\t')
    # We use "tab" as the "start sequence" character
    # for the targets, and "\n" as "end sequence" character.
    target_text = '\t' + target_text + '\n'
    input_texts.append(input_text)
    target_texts.append(target_text)
    #文から単語を抽出し、単語のsetに格納
    for char in input_text:
        if char not in input_characters:
            input_characters.add(char)
    for char in target_text:
        if char not in target_characters:
            target_characters.add(char)
#単語のsetをリスト化し、ソート
input_characters = sorted(list(input_characters))
target_characters = sorted(list(target_characters))

#エンコーダの特徴量の数を一意の単語の数に設定
num_encoder_tokens = len(input_characters)
num_decoder_tokens = len(target_characters)

#文の数からデータ数を設定
max_encoder_seq_length = max([len(txt) for txt in input_texts])
max_decoder_seq_length = max([len(txt) for txt in target_texts])

#情報を印刷してデバッグ
print('Number of samples:', len(input_texts))
print('Number of unique input tokens:', num_encoder_tokens)
print('Number of unique output tokens:', num_decoder_tokens)
print('Max sequence length for inputs:', max_encoder_seq_length)
print('Max sequence length for outputs:', max_decoder_seq_length)

#単語ごとにインデックスを紐付ける
input_token_index = dict(
    [(char, i) for i, char in enumerate(input_characters)])
target_token_index = dict(
    [(char, i) for i, char in enumerate(target_characters)])

#エンコーダデコーダそれぞれにinputするデータと教師データの型となる変数を定義
encoder_input_data = np.zeros(
    (len(input_texts), max_encoder_seq_length, num_encoder_tokens),
    dtype='float32')
decoder_input_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')
decoder_target_data = np.zeros(
    (len(input_texts), max_decoder_seq_length, num_decoder_tokens),
    dtype='float32')

#for文で、文入力し、教師データと比較して学習していく
for i, (input_text, target_text) in enumerate(zip(input_texts, target_texts)):
    #文から単語を抽出
    for t, char in enumerate(input_text):
        #読み取ったデータの情報とデータの型を元にベクトル化。(i番目文, t番目単語, 単語ベクトル)
        encoder_input_data[i, t, input_token_index[char]] = 1.
    #最後に空白を挿入
    encoder_input_data[i, t + 1:, input_token_index[' ']] = 1.
    #同様の処理を教デコード用データ、教師データにも実施。
    #この時教師データがデコーダデータの一つあとの単語のデータであることを考慮
    for t, char in enumerate(target_text):
        # decoder_target_data is ahead of decoder_input_data by one timestep
        decoder_input_data[i, t, target_token_index[char]] = 1.
        if t > 0:
            # decoder_target_data will be ahead by one timestep
            # and will not include the start character.
            decoder_target_data[i, t - 1, target_token_index[char]] = 1.
    decoder_input_data[i, t + 1:, target_token_index[' ']] = 1.
    decoder_target_data[i, t:, target_token_index[' ']] = 1.
# Define an input sequence and process it.

#seq2seqモデルの定義
encoder_inputs = Input(shape=(None, num_encoder_tokens))
encoder = LSTM(latent_dim, return_state=True)
encoder_outputs, state_h, state_c = encoder(encoder_inputs)
# We discard `encoder_outputs` and only keep the states.
encoder_states = [state_h, state_c]

# Set up the decoder, using `encoder_states` as initial state.
decoder_inputs = Input(shape=(None, num_decoder_tokens))
# We set up our decoder to return full output sequences,
# and to return internal states as well. We don't use the
# return states in the training model, but we will use them in inference.
decoder_lstm = LSTM(latent_dim, return_sequences=True, return_state=True)
decoder_outputs, _, _ = decoder_lstm(decoder_inputs,
                                     initial_state=encoder_states)
decoder_dense = Dense(num_decoder_tokens, activation='softmax')
decoder_outputs = decoder_dense(decoder_outputs)

# Define the model that will turn
# `encoder_input_data` & `decoder_input_data` into `decoder_target_data`
model = Model([encoder_inputs, decoder_inputs], decoder_outputs)

#モデルのコンパイル
# Run training
model.compile(optimizer='rmsprop', loss='categorical_crossentropy',
              metrics=['accuracy'])
#学習スタート
model.fit([encoder_input_data, decoder_input_data], decoder_target_data,
          batch_size=batch_size,
          epochs=epochs,
          validation_split=0.2)
#モデルを保存
# Save model
model.save('s2s.h5')

# Next: inference mode (sampling).
# Here's the drill:
# 1) encode input and retrieve initial decoder state
# 2) run one step of decoder with this initial state
# and a "start of sequence" token as target.
# Output will be the next target token
# 3) Repeat with the current target token and current states

#予測用モデルを定義
# Define sampling models
encoder_model = Model(encoder_inputs, encoder_states)

decoder_state_input_h = Input(shape=(latent_dim,))
decoder_state_input_c = Input(shape=(latent_dim,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(
    decoder_inputs, initial_state=decoder_states_inputs)
decoder_states = [state_h, state_c]
decoder_outputs = decoder_dense(decoder_outputs)
decoder_model = Model(
    [decoder_inputs] + decoder_states_inputs,
    [decoder_outputs] + decoder_states)

# Reverse-lookup token index to decode sequences back to
# something readable.
reverse_input_char_index = dict(
    (i, char) for char, i in input_token_index.items())
reverse_target_char_index = dict(
    (i, char) for char, i in target_token_index.items())

#文を引数で受け取り、それを元に隠れ状態ベクトルを生成し、それを元に、　文章を生成する関数を定義
def decode_sequence(input_seq):
    # Encode the input as state vectors.
    states_value = encoder_model.predict(input_seq)

    # Generate empty target sequence of length 1.
    target_seq = np.zeros((1, 1, num_decoder_tokens))
    # Populate the first character of target sequence with the start character.
    target_seq[0, 0, target_token_index['\t']] = 1.

    # Sampling loop for a batch of sequences
    # (to simplify, here we assume a batch of size 1).
    stop_condition = False
    decoded_sentence = ''
    while not stop_condition:
        output_tokens, h, c = decoder_model.predict(
            [target_seq] + states_value)

        # Sample a token
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = reverse_target_char_index[sampled_token_index]
        decoded_sentence += sampled_char

        # Exit condition: either hit max length
        # or find stop character.
        if (sampled_char == '\n' or
           len(decoded_sentence) > max_decoder_seq_length):
            stop_condition = True

        # Update the target sequence (of length 1).
        target_seq = np.zeros((1, 1, num_decoder_tokens))
        target_seq[0, 0, sampled_token_index] = 1.

        # Update states
        states_value = [h, c]

    return decoded_sentence


#実際に文章生成を行うfor文を回す。
for seq_index in range(100):
    # Take one sequence (part of the training set)
    # for trying out decoding.
    input_seq = encoder_input_data[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print('-')
    print('Input sentence:', input_texts[seq_index])
    print('Decoded sentence:', decoded_sentence)

Number of samples: 10000
Number of unique input tokens: 70
Number of unique output tokens: 93
Max sequence length for inputs: 16
Max sequence length for outputs: 59
Train on 8000 samples, validate on 2000 samples
Epoch 1/10
8000/8000 [==============================] - 47s 6ms/step - loss: 1.2073 - acc: 0.7231 - val_loss: 1.0723 - val_acc: 0.7000
Epoch 2/10
8000/8000 [==============================] - 45s 6ms/step - loss: 0.8559 - acc: 0.7679 - val_loss: 0.8671 - val_acc: 0.7615
Epoch 3/10
8000/8000 [==============================] - 43s 5ms/step - loss: 0.6734 - acc: 0.8090 - val_loss: 0.7273 - val_acc: 0.7890
Epoch 4/10
8000/8000 [==============================] - 46s 6ms/step - loss: 0.5914 - acc: 0.8282 - val_loss: 0.6529 - val_acc: 0.8097
Epoch 5/10
8000/8000 [==============================] - 43s 5ms/step - loss: 0.5425 - acc: 0.8407 - val_loss: 0.6126 - val_acc: 0.8205
Epoch 6/10
8000/8000 [==============================] - 42s 5ms/step - loss: 0.5049 - acc: 0.8513 - val_loss: 0.

//anaconda3/envs/aistudio/lib/python3.6/site-packages/keras/engine/network.py:877: UserWarning: Layer lstm_4 was passed non-serializable keyword arguments: {'initial_state': [<tf.Tensor 'lstm_3/while/Exit_2:0' shape=(?, 256) dtype=float32>, <tf.Tensor 'lstm_3/while/Exit_3:0' shape=(?, 256) dtype=float32>]}. They will not be included in the serialized model (and thus will be missing at deserialization time).
  '. They will not be included '


-
Input sentence: Go.
Decoded sentence: Dentez-vous !

-
Input sentence: Hi.
Decoded sentence: Sois sestie !

-
Input sentence: Hi.
Decoded sentence: Sois sestie !

-
Input sentence: Run!
Decoded sentence: Dentez-vous !

-
Input sentence: Run!
Decoded sentence: Dentez-vous !

-
Input sentence: Who?
Decoded sentence: Est-ce que j'ai pas ?

-
Input sentence: Wow!
Decoded sentence: Dentez-vous !

-
Input sentence: Fire!
Decoded sentence: Dentez-vous !

-
Input sentence: Help!
Decoded sentence: Parte le moi.

-
Input sentence: Jump.
Decoded sentence: Dentez-vous !

-
Input sentence: Stop!
Decoded sentence: Reste de chanter.

-
Input sentence: Stop!
Decoded sentence: Reste de chanter.

-
Input sentence: Stop!
Decoded sentence: Reste de chanter.

-
Input sentence: Wait!
Decoded sentence: Dentez-vous !

-
Input sentence: Wait!
Decoded sentence: Dentez-vous !

-
Input sentence: Go on.
Decoded sentence: Vais entre !

-
Input sentence: Go on.
Decoded sentence: Vais entre !

-
Input sentence: Go 

In [8]:
!pwd

/Users/shigaryouta/diveintocode-ml/sprint24


In [16]:
! ls ../.././Downloads//fra-eng/

_about.txt fra.txt


In [41]:
! ls /Users/shigaryouta/Downloads/fra-eng/

_about.txt fra.txt


## 問題１のコードの説明はコード内の#で記載。

In [46]:
from __future__ import print_function
from keras.models import Model
from keras.layers import Input, LSTM, Dense
import numpy as np

batch_size = 64
epochs=100
latent_dim = 256
num_samples = 10000

data_path = '/Users/shigaryouta/Downloads/fra-eng/fra.txt'

input_texts = []
target_texts = []
input_characters = set()
target_characters = set()

#utf-8でreadモードで読み込み
with open(data_path, 'r', encoding='utf-8') as f:
    lines = f.read().split('\n')#改行を削除
    count = 0
    for line in lines[: min(num_samples, len(lines) - 1)]:
        print('line: ', line)
        count += 1
        input_text, target_text, _ = line.split('\t')
        print('input_text: ', input_text)
        print('target_text: ', target_text)
        if count == 5:
            break
    

line:  Go.	Va !	CC-BY 2.0 (France) Attribution: tatoeba.org #2877272 (CM) & #1158250 (Wittydev)
input_text:  Go.
target_text:  Va !
line:  Hi.	Salut !	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #509819 (Aiji)
input_text:  Hi.
target_text:  Salut !
line:  Hi.	Salut.	CC-BY 2.0 (France) Attribution: tatoeba.org #538123 (CM) & #4320462 (gillux)
input_text:  Hi.
target_text:  Salut.
line:  Run!	Cours !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906331 (sacredceltic)
input_text:  Run!
target_text:  Cours !
line:  Run!	Courez !	CC-BY 2.0 (France) Attribution: tatoeba.org #906328 (papabear) & #906332 (sacredceltic)
input_text:  Run!
target_text:  Courez !


# 問題2 イメージキャプショニングの学習済みモデルの実行

sprint24_monndai2参照

# 問題3 Kerasで動かしたい場合はどうするかを調査

相互変換ツールMMdnnを使用する。
1.  pip install mmdnnでインストール 
2.  mmdownload -f pytorchでpytorchのモデルをダウンロード
3.  mmdownload -f pytorch -n inception_v3 -o ./　で例えばinception_v3モデルをカレントディレクトリにダウンロード。imagenet_inception_v3.pthという名前で保存される。
4.  mmconvert -sf pytorch -iw imagenet_inception_v3.pth -df keras -om imagenet_inception_v3.h5でモデルを変換する
